In [ ]:
# pip install requests
# pip install beautifulsoup4
# pip install nltk 
# pip install spacy
# pip install textblob
# pip install wordcloud

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL des Wahlprogramms
url = "https://www.die-linke.de/bundestagswahl-2025/wahlprogramm/"

# HTTP-Request senden
response = requests.get(url)

# Überprüfen, ob der Request erfolgreich war
if response.status_code == 200:
    # HTML-Inhalt parsen
    soup = BeautifulSoup(response.content, 'html.parser')

    # Den Hauptinhalt der Seite extrahieren (je nach Struktur der Website anpassen)
    main_content = soup.find('div', class_='ce-bodytext')  # Beispiel für typische Content-Div-Klasse

    if main_content:
        # Text bereinigen und ausgeben
        text = main_content.get_text(separator='\n', strip=True)
        print(text)
        
        # Optional: in eine Textdatei speichern
        with open('wahlprogramm_die_linke_2025.txt', 'w', encoding='utf-8') as f:
            f.write(text)
    else:
        print("Hauptinhalt nicht gefunden. Bitte die Struktur der Website überprüfen.")
else:
    print(f"Fehler beim Abrufen der Seite: {response.status_code}")
